In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import requests
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from IPython.display import display

In [2]:
file_path = "hotels_pages\\Booking2.html"
with open(file_path, 'r', encoding='utf-8') as file:
    content = file.read()

soup = BeautifulSoup(content, 'html.parser')

In [3]:
hotels=soup.find_all('div',{'class':'c1edfbabcb'})

In [4]:
hotel_name=[]
dis_beach=[]
grade=[]
location_score=[]
dis_center=[]
price=[]
opinion_number=[]
linke=[]

In [5]:
for hotel in hotels:
    hotel_name.append(hotel.find('div',{'class':'f6431b446c a15b38c233'}).get_text())
    num_pf_opin=hotel.find('div',{'class':'abf093bdfe f45d8e4c32 d935416c47'}).get_text()
    opinion_number.append((re.findall("(.*).חוות דעת",num_pf_opin)[0]).replace(',', ''))
    center=hotel.find('span',{'data-testid':"distance"}).get_text()
    if "ק''מ" in center:
        dis_center.append(float(re.findall("(.*).+ק''מ",center)[0])*1000)
    else:
        if "מול" in center:
            dis_center.append(0)
        else:
            dis_center.append(float(re.findall("(.*).מ'",center)[0]))
    beach=hotel.find('span',{'class':'abf093bdfe b058f54b9a'}).get_text()
    if "ק''מ" in beach:
        dis_beach.append(float(re.findall("(.*).+ק''מ",beach)[0])*1000)
    else:
        if "מול" in beach:
            dis_beach.append(0)
        else:
            dis_beach.append(float(re.findall("(.*).מ'",beach)[0]))
    grade.append(float(hotel.find('div',{'class':'a3b8729ab1 d86cee9b25'}).get_text()))
    location_score.append(float(hotel.find('div',{'class':'a3b8729ab1 d86cee9b25'}).get_text()))
    price.append(int(''.join(re.findall(r'\d',hotel.find('span',{'class':'f6431b446c fbfd7c1165 e84eb96b1f'}).get_text()))))

In [6]:
df={"hotel_name":hotel_name,"Dist_from_the_beach":dis_beach,"Dist_from_the_center":dis_center,"price":price,"opinion_number":opinion_number,"location_score":location_score,"grade":grade}
df = pd.DataFrame(df)

In [7]:
#links=["מלון בוטיק דיזנגוף אווניו.html","Beach Boutique Hotel And Spa By TLV hotels.html"]
links=["מלון בוטיק דיזנגוף אווניו.html","Beach Boutique Hotel And Spa By TLV hotels.html","מלון BY14 TLV.html","דירות רוק 130.html","סלינה פרישמן תל אביב.html","בראון ברוט Seafront.html","מלון פרימה סיטי.html","Mulan TLV Hotel.html","H7 TLV.html","קראון פלזה סיטי סנטר תל אביב.html","פרימה תל אביב.html","מלון אמבסי תל אביב,.html","Assemblage Boutique.html","מלון סבוי תל אביב.html","סלינה חוף תל אביב.html","מלון בוטיק אולטרה תל אביב.html","פלורנטין האוס.html","מלון אינדיגו תל אביב - מתחם הבורסה ליהלומים.html","Abraham Tel Aviv.html","הוטל 75 מרשת מלונות פרימה.html","דיזינגוף אין אפרטמנטס.html","Link Hotel & Hub מרשת מלונות דן.html","סי נט מבית AFI HOTELS.html","מלון אברטל סוויטס.html","סאן סיטי.html"]

In [8]:
name_id=[]
location_hotel=[]
team_score=[]
comfort_score=[]
score_internet=[]
facility_spec=[]
score_equal_price=[]
cleanliness_score=[]
opinions={}

In [9]:
for link in links:
    file_path = "hotels_pages\\" + link
    
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    soup = BeautifulSoup(content, 'html.parser')
    
    hotel_pages = soup.find_all('div', {"id": "hotelTmpl"})
    name_of_hotel=soup.find('h2', {"class": "d2fee87262 pp-header__title"}).get_text()
    name_id.append(name_of_hotel)
#location
    for hotel_page in hotel_pages:
        location_spans = hotel_page.find_all('span', {"data-node_tt_id": "location_score_tooltip"})

        for location_span in location_spans:
            location_hotel.append(location_span.get_text().strip())
#scores
    categorys=soup.find('div', {"class":"aca0ade214 ebac6e22e9 f66f916626 c2931f4182 c27e5d305d db150fece4"})
    for cat in categorys:
        scores=cat.get_text()
        if "צוות" in scores:
            team_score.append(float(re.findall("צוות.(.*)",scores)[0]))
        elif "מתקנים" in scores:
            facility_spec.append(float(re.findall("מתקנים.(.*)",scores)[0]))
        elif "ניקיון" in scores:
            cleanliness_score.append(float(re.findall("ניקיון.(.*)",scores)[0]))
        elif "נוחות" in scores:
            comfort_score.append(float(re.findall("נוחות.(.*)",scores)[0]))
        elif "תמורה למחיר" in scores:
            score_equal_price.append(float(re.findall("תמורה למחיר.(.*)",scores)[0]))
        elif "אינטרנט אלחוטי חינם" in scores:
            score_internet.append(float(re.findall("אינטרנט אלחוטי חינם.(.*)",scores)[0]))


In [10]:
df_to_marg={"hotel_name":name_id,"Address":location_hotel,"team_score":team_score,"comfort_score":comfort_score,"score_internet":score_internet,"facility_spec":facility_spec,"score_equal_price":score_equal_price,"cleanliness_score":cleanliness_score}
df_to_marg = pd.DataFrame(df_to_marg)

In [11]:
names=df["hotel_name"].tolist()
to_marg=df_to_marg["hotel_name"].tolist()

In [12]:
difference_list1 = [item for item in names if item not in to_marg]
difference_list2 = [item for item in to_marg if item not in names]

In [13]:
data= pd.merge(df, df_to_marg, on='hotel_name', how='left')

In [14]:
names=df["hotel_name"].tolist()
to_marg=df_to_marg["hotel_name"].tolist()

In [15]:
for name in names:
    if name not in to_marg:
        print(name)

In [16]:
data.to_csv('Hotel_details.csv', index=False)

In [17]:
opinion_df = pd.DataFrame(opinions)

In [18]:
#Finish pulling data - start analysis

In [19]:
links=["חווד סאן סיטי.html","חווד דירות רוק 130.html","סי נט מבית AFI HOTELS חווד.html","מלון BY14 TLV חווד.html","חווד מלון אינדיגו תל אביב.html","Mulan TLV Hotel חווד.html","חווד פלורנטין האוס.html","חווד בראון ברוט Seafront.html","חווד דיזינגוף אין אפרטמנטס.html","חווד Hub מרשת מלונות דן.html","Abraham Tel Aviv חווד.html","Beach Boutique Hotel And Spa By TLV hotels חווד.html","חווד סלינה חוף תל אביב.html","חווד קראון פלזה סיטי סנטר תל אביב.html","H7 TLV חווד.html","חווד סלינה פרישמן תל אביב.html","חווד הוטל 75 מרשת מלונות פרימה.html","חווד מלון סבוי תל אביב.html","חווד מלון בוטיק אולטרה תל אביב.html","חווד מלון פרימה סיטי.html","חווד מלון אמבסי תל אביב.html","Assemblage Boutique חווד.html","חווד מלון אברטל סוויטס.html","חווד פרימה תל אביב.html","חווד מלון בוטיק דיזנגוף אווניו.html"]

In [20]:
opinion_hotel=[]
for link in links:
    file_path = "hotels_pages\\"+link
    
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    soup = BeautifulSoup(content, 'html.parser')
    opinions_hotl_name=soup.find_all('h2', {"class": "d2fee87262 pp-header__title"})
    for name in opinions_hotl_name:
        opinion_hotel.append(name.get_text())

In [21]:
index_name = 0
opinions2 = {} 
for link in links:
    file_path = "hotels_pages\\" + link

    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    soup = BeautifulSoup(content, 'html.parser')
    opinions_all = soup.find_all('div', {"class": "c-review"})
    name = opinion_hotel[index_name]

    opinions_list = [] 
    for opin in opinions_all:
        if opin is not np.nan:
            to_enter=opin.get_text().replace('\n', '').replace('אהבו', '').replace('לא אהבו', '').replace('מתרגם... - הצג מקורהצגת התרגום', '').replace('\xa0', '')
            opinions_list.append(to_enter.replace('לא', '').replace('·', ''))
        else:
            opinions_list.append("")

    opinions2[name] = opinions_list
    index_name += 1

In [22]:
opinion_df = pd.DataFrame(opinions2)
opinion_df.to_csv('opinions_hotel.csv', index=False)